In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time as t
import requests
import csv
from selenium import webdriver
from pprint import pprint
from scraper import scrape_site
from pathlib import Path
import pandas as pd
from config import subscription_key
from urllib.error import HTTPError
from googleapi import google
from urllib.parse import urlencode, urlparse, parse_qs, urlsplit

import requests.exceptions
from collections import deque

from lxml.html import fromstring
from requests import get

In [2]:
file_to_output = Path("Data_Scraped/Contacts_Final.csv")

In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', headless=False, **executable_path)

In [4]:
url = 'https://www.bizjournals.com/houston/potm?l=&time=&type=&ind=17'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
Final_List = []
dict_list =[]
links = []

In [13]:

for link in soup.find_all("a", class_="item", href=True):
        links.append(link.get('href'))
#Get Next button
for link in soup.find_all("a", class_="btn btn--primary pull-right", href=True):
    button = link.get('href')


for i in links:
    if '/houston/potmsearch/detail/submission' in i:
        d = {}
        link = "https://www.bizjournals.com/" + i
        newI = i.split("?")
        slicefront = (newI[0])
        name = slicefront[46:]
        final_name = name.replace('_', " ")
        d["Name"] = final_name
        d["Link"] = link
        if d not in dict_list:
            dict_list.append(d)
        else:
            pass
    else:
        pass

In [14]:

try:
#     browser.click_link_by_partial_href('Next')
    browser.click_link_by_href(button) 
#     browser.find_link_by_text('Next').first.click()
#     browser.links.find_by_partial_text("Next").click()
    print("Went to the next page")
    html = ''
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
except:
    print("Finished all pages")


Finished all pages


In [15]:
len(dict_list)

99

In [25]:
pprint(dict_list)

[{'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6490696/Charles_Harrell?l=&time=&ind=17&type=&ro=0',
  'Name': 'Charles Harrell'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489778/Bryce_Kaufman?l=&time=&ind=17&type=&ro=1',
  'Name': 'Bryce Kaufman'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489777/Kevin_Richardson?l=&time=&ind=17&type=&ro=2',
  'Name': 'Kevin Richardson'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489683/Zach_Burford?l=&time=&ind=17&type=&ro=3',
  'Name': 'Zach Burford'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489678/Laura_S_Rahman?l=&time=&ind=17&type=&ro=4',
  'Name': 'Laura S Rahman'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489465/Daniel_Dutko?l=&time=&ind=17&type=&ro=5',
  'Name': 'Daniel Dutko'},
 {'Link': 'https://www.bizjournals.com//houston/potmsear

In [16]:
# browser.visit("https://www.bizjournals.com/houston/potmsearch/detail/submission/6488386/Charles_M_Rosson?time=most_recent&ind=86&type=all&id=&ro=1")
# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')
# date_info= soup.find_all('span', class_='meta-item')
# type_info= soup.find_all('a', class_='meta-item')
# info_list = soup.find_all('p', class_ = 'cXenseParse')
# education = soup.find_all("span", class_ = "cXenseParse")
# print(education[0].text)
# # print(type_info[0].text)
# # print(type_info[1].text)
# # print(date_info[0].text)

In [17]:
for i in dict_list:

        try:
                browser.visit(i["Link"])
                t.sleep(1)
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                info_list = soup.find_all('p', class_ = 'cXenseParse')
                date_info = soup.find_all('span', class_='meta-item')
                type_info= soup.find_all('a', class_='meta-item')
                education = soup.find_all("span", class_ = "cXenseParse")
                try: 
                    i["Title"] = info_list[0].text
                    i["Blurb"] = info_list[1].text
                    i["Publish Date"] = date_info[0].text
                    i["New Hire/Promotion"] = type_info[0].text
                    i["Industry"] = type_info[1].text
                    i["Education"] = education[0].text
                except:
                    print(f"Missing info for {i['Name']}")
                Final_List.append(i)

        except HTTPError as error:
            print(error)
length_Final_List = len(Final_List)
print(f"Finished pulling. {length_Final_List} items pulled")

Missing info for Laura S Rahman
Missing info for Andrea Hellyer
Missing info for Kathlyn Hufstetler
Missing info for Michael L Telford
Missing info for Stephen Jacobson
Missing info for Loren Canales
Missing info for Sara Valentine
Missing info for Wendy Burgower
Missing info for Vanessa Lee Pierce
Missing info for Randy SorrelsPresident State Bar of Texas
Missing info for Liz Klingensmith
Finished pulling. 99 items pulled


In [18]:
len(Final_List)

99

In [19]:
pprint(Final_List)

[{'Blurb': 'Corporate attorney Charles Harrell has joined the Corporate & '
           'Securities practice in the Houston office of Jackson Walker LLP. '
           'With decades of experience, he has guided businesses ranging from '
           'technology, healthcare, and energy startups to mid-market\n'
           'and Fortune 100 companies through all types of corporate matters. '
           'His focus is on\n'
           'complex transactions including mergers & acquisitions, '
           'cross-border matters, finance and capital markets. He also advises '
           'board committees on corporate governance.',
  'Education': "EDUCATION:\xa0\xa0St. Mary's University School of Law",
  'Industry': '\n'
              '                                        Law '
              'Practice                                    ',
  'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6490696/Charles_Harrell?l=&time=&ind=17&type=&ro=0',
  'Name': 'Charles Harrell',
  

  'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6479417/Mark_Smith?l=&time=&ind=17&type=&ro=58',
  'Name': 'Mark Smith',
  'New Hire/Promotion': '\nNew Hire\n',
  'Publish Date': 'Dec 18, 2020',
  'Title': 'Partner at Thompson & Horton LLP'},
 {'Blurb': 'Ducran works closely with general contractors, owners, architects, '
           'and subcontractors to provide practical counsel regarding the '
           'avoidance and resolution of disputes on construction projects. He '
           'offers extensive industry experience in preparing and negotiating '
           'contracts, litigation, arbitration and mediation of construction '
           'disputes. He is Board Certified in Construction Law by the Texas '
           'Board of Legal Specialization and is a registered architect.',
  'Education': 'EDUCATION:\xa0\xa0South Texas College of Law (Houston, TX)',
  'Industry': '\n'
              '                                        Legal '
              'Serv

In [20]:
assert subscription_key

In [21]:
for i in Final_List:
    try:
        search_url = "https://api.bing.microsoft.com/v7.0/search"
        search_term = i["Name"] + ' ' + i["Title"]
        headers = {"Ocp-Apim-Subscription-Key": subscription_key}
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get(search_url, headers=headers, params=params)
        response.raise_for_status()
        search_results = response.json()
        i['Bing Search Link'] = search_results['webPages']['value'][0]['url']
        print(f"Found {i['Name']}!")
    except:
        print(f"Could not find {i['Name']}")
        print(subscription_key)
        print(response)
    t.sleep(1)

Found Charles Harrell!
Found Bryce Kaufman!
Found Kevin Richardson!
Found Zach Burford!
Found Laura S Rahman!
Found Daniel Dutko!
Found Bethany Spare!
Found Andrea Hellyer!
Found Adrienne Randle Bond!
Found Eric Thiergod Sr!
Found Charles M Rosson!
Found Heath Trisdale!
Found Michael Hamilton!
Found Travis Clardy!
Found David Peterman!
Found Jonathan Newton!
Found John Farrell!
Found Connie Pfeiffer!
Found Kathlyn Hufstetler!
Found Sammy Ford IV!
Found Colleen Migl!
Found Brooks Tueting!
Found Edward Ed L Ripley!
Found Stephanie McGraw!
Found Ben Walther!
Found Michael L Telford!
Found Elizabeth Liz Webb!
Found Caroline C Pace!
Found Keith A Taylor!
Found Gabriela Gabby M Barake!
Found Stephen Jacobson!
Found Justo Mendez!
Found Samuel Louis!
Found S Priya Coffey!
Found Joe Flack!
Found Christopher Hogan!
Found Samantha Thompson!
Found David W Parker!
Found Kristhy Peguero!
Found William AI Trey McDonald III!
Found Briggs Knight!
Found Kate Kaufmann Shih!
Found Lindsey Raspino!
Found K

In [30]:
Final_List

[{'Name': 'Charles Harrell',
  'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6490696/Charles_Harrell?l=&time=&ind=17&type=&ro=0',
  'Title': 'Partner at Jackson Walker LLP',
  'Blurb': 'Corporate attorney Charles Harrell has joined the Corporate & Securities practice in the Houston office of Jackson Walker LLP. With decades of experience, he has guided businesses ranging from technology, healthcare, and energy startups to mid-market\nand Fortune 100 companies through all types of corporate matters. His focus is on\ncomplex transactions including mergers & acquisitions, cross-border matters, finance and capital markets. He also advises board committees on corporate governance.',
  'Publish Date': 'Dec 18, 2020',
  'New Hire/Promotion': '\nNew Hire\n',
  'Industry': '\n                                        Law Practice                                    ',
  'Education': "EDUCATION:\xa0\xa0St. Mary's University School of Law",
  'Bing Search Link': 'https:/

In [ ]:
# for i in Final_List:
#     emails = set()
#     try:
#         response = requests.get(i["Bing Search Link"])
#         new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
#         emails.update(new_emails)
#         i['Bing Emails'] = emails
#         if emails == set():
#             print(f'Set empty on {i["Name"]}')
#             emails = set()
#         else:
#             print(f"Successfully captured an email for {i['Name']}")
#             emails = set()
#     except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
#         # ignore pages with errors and continue with next url
#         print(f"Could not find {i['Name']}")
#         continue
# print(f"Finished all email grabs")

In [65]:
FL = pd.DataFrame(Final_List)
FL.head()

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Industry,Education
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"December 7, 2020",\nNew Hire\n,\n Law ...,EDUCATION: St. Mary's University School of Law
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"November 10, 2020",\nPromotion\n,\n Law ...,EDUCATION: Stanford University Law School
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"November 10, 2020",\nPromotion\n,\n Law ...,EDUCATION: University of Texas School of Law
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"November 4, 2020",\nNew Hire\n,\n Law ...,EDUCATION: The University of Texas School of ...
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"November 4, 2020",\nNew Hire\n,\n Law ...,NaN


In [155]:
FL["New Hire/Promotion"] = FL["New Hire/Promotion"].str.strip('\n')
FL["Industry"] = FL["Industry"].str.strip('\n')
FL["Blurb"] = FL["Blurb"].str.strip('\n')
FL["Blurb"] = FL["Blurb"].replace(r'\\n',' ', regex=True) 
FL["Blurb"] = FL["Blurb"].replace('\n','', regex=True)
FL["Blurb"] = FL["Blurb"].replace('\n\n',' ', regex=True)
FL["Industry"] = FL["Industry"].str.strip()
FL["Education"] = FL["Education"].str.strip('EDUCATION:  ')
FL["Education"] = FL["Education"].str.strip()
FL.head()

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Industry,Education,First Name,Last Name
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"December 7, 2020",New Hire,Law Practice,St. Mary's University School of Law,Charles,Harrell
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"November 10, 2020",Promotion,Law Practice,Stanford University Law School,Bryce,Kaufman
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"November 10, 2020",Promotion,Law Practice,niversity of Texas School of Law,Kevin,Richardson
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"November 4, 2020",New Hire,Law Practice,"he University of Texas School of Law, Doctoral...",Zach,Burford
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"November 4, 2020",New Hire,Law Practice,NaN,Laura,S Rahman


In [156]:
pd.set_option('display.max_rows', None)
FL

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Industry,Education,First Name,Last Name
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"December 7, 2020",New Hire,Law Practice,St. Mary's University School of Law,Charles,Harrell
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"November 10, 2020",Promotion,Law Practice,Stanford University Law School,Bryce,Kaufman
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"November 10, 2020",Promotion,Law Practice,niversity of Texas School of Law,Kevin,Richardson
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"November 4, 2020",New Hire,Law Practice,"he University of Texas School of Law, Doctoral...",Zach,Burford
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"November 4, 2020",New Hire,Law Practice,NaN,Laura,S Rahman
5,Daniel Dutko,https://www.bizjournals.com//houston/potmsearc...,"Partner at Rusty Hardin & Associates, LLP",Mr. Dutko is a seasoned trial lawyer whose pra...,"October 29, 2020",Promotion,Law Practice,"South Texas College of Law (Houston, TX)",Daniel,Dutko
6,Bethany Spare,https://www.bizjournals.com//houston/potmsearc...,Associate at Yetter Coleman LLP,Bethany Spare represents clients in complex co...,"October 29, 2020",New Hire,Legal Services,he University of Texas School of Law,Bethany,Spare
7,Andrea Hellyer,https://www.bizjournals.com//houston/potmsearc...,"Director Client Development at Knightscope, Inc.","Andrea Hellyer joined Knightscope, Inc. in Oct...","October 27, 2020",New Hire,Security & Investigations,NaN,Andrea,Hellyer
8,Adrienne Randle Bond,https://www.bizjournals.com//houston/potmsearc...,Law Firm Shareholder at Crain Caton & James PC,A seasoned attorney whose dynamic career spans...,"October 13, 2020",New Hire,Law Practice,olumbia School of Law,Adrienne,Randle Bond
9,Eric Thiergod Sr,https://www.bizjournals.com//houston/potmsearc...,Partner at Kean Miller LLP,Eric is a corporate attorney whose practice in...,"October 6, 2020",New Hire,Law Practice,"South Texas College of Law (Houston, TX)",Eric,Thiergod Sr


In [157]:
FL.isna().sum()

Name                   0
Link                   0
Title                  0
Blurb                  2
Publish Date           2
New Hire/Promotion     2
Industry               2
Education             22
First Name             0
Last Name              0
dtype: int64

In [158]:
new = FL["Name"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
FL["First Name"]= new[0] 
  
# making separate last name column from new data frame 
FL["Last Name"]= new[1]
FL

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Industry,Education,First Name,Last Name
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"December 7, 2020",New Hire,Law Practice,St. Mary's University School of Law,Charles,Harrell
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"November 10, 2020",Promotion,Law Practice,Stanford University Law School,Bryce,Kaufman
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"November 10, 2020",Promotion,Law Practice,niversity of Texas School of Law,Kevin,Richardson
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"November 4, 2020",New Hire,Law Practice,"he University of Texas School of Law, Doctoral...",Zach,Burford
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"November 4, 2020",New Hire,Law Practice,NaN,Laura,S Rahman
5,Daniel Dutko,https://www.bizjournals.com//houston/potmsearc...,"Partner at Rusty Hardin & Associates, LLP",Mr. Dutko is a seasoned trial lawyer whose pra...,"October 29, 2020",Promotion,Law Practice,"South Texas College of Law (Houston, TX)",Daniel,Dutko
6,Bethany Spare,https://www.bizjournals.com//houston/potmsearc...,Associate at Yetter Coleman LLP,Bethany Spare represents clients in complex co...,"October 29, 2020",New Hire,Legal Services,he University of Texas School of Law,Bethany,Spare
7,Andrea Hellyer,https://www.bizjournals.com//houston/potmsearc...,"Director Client Development at Knightscope, Inc.","Andrea Hellyer joined Knightscope, Inc. in Oct...","October 27, 2020",New Hire,Security & Investigations,NaN,Andrea,Hellyer
8,Adrienne Randle Bond,https://www.bizjournals.com//houston/potmsearc...,Law Firm Shareholder at Crain Caton & James PC,A seasoned attorney whose dynamic career spans...,"October 13, 2020",New Hire,Law Practice,olumbia School of Law,Adrienne,Randle Bond
9,Eric Thiergod Sr,https://www.bizjournals.com//houston/potmsearc...,Partner at Kean Miller LLP,Eric is a corporate attorney whose practice in...,"October 6, 2020",New Hire,Law Practice,"South Texas College of Law (Houston, TX)",Eric,Thiergod Sr


In [160]:
FL_names = FL.drop(columns =["Name"])
FL_names

,Link,Title,Blurb,Publish Date,New Hire/Promotion,Industry,Education,First Name,Last Name
0,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"December 7, 2020",New Hire,Law Practice,St. Mary's University School of Law,Charles,Harrell
1,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"November 10, 2020",Promotion,Law Practice,Stanford University Law School,Bryce,Kaufman
2,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"November 10, 2020",Promotion,Law Practice,niversity of Texas School of Law,Kevin,Richardson
3,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"November 4, 2020",New Hire,Law Practice,"he University of Texas School of Law, Doctoral...",Zach,Burford
4,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"November 4, 2020",New Hire,Law Practice,NaN,Laura,S Rahman
5,https://www.bizjournals.com//houston/potmsearc...,"Partner at Rusty Hardin & Associates, LLP",Mr. Dutko is a seasoned trial lawyer whose pra...,"October 29, 2020",Promotion,Law Practice,"South Texas College of Law (Houston, TX)",Daniel,Dutko
6,https://www.bizjournals.com//houston/potmsearc...,Associate at Yetter Coleman LLP,Bethany Spare represents clients in complex co...,"October 29, 2020",New Hire,Legal Services,he University of Texas School of Law,Bethany,Spare
7,https://www.bizjournals.com//houston/potmsearc...,"Director Client Development at Knightscope, Inc.","Andrea Hellyer joined Knightscope, Inc. in Oct...","October 27, 2020",New Hire,Security & Investigations,NaN,Andrea,Hellyer
8,https://www.bizjournals.com//houston/potmsearc...,Law Firm Shareholder at Crain Caton & James PC,A seasoned attorney whose dynamic career spans...,"October 13, 2020",New Hire,Law Practice,olumbia School of Law,Adrienne,Randle Bond
9,https://www.bizjournals.com//houston/potmsearc...,Partner at Kean Miller LLP,Eric is a corporate attorney whose practice in...,"October 6, 2020",New Hire,Law Practice,"South Texas College of Law (Houston, TX)",Eric,Thiergod Sr


In [161]:
# Reorder columns
FL_names = FL_names[["First Name", "Last Name", "Title", "Publish Date", "New Hire/Promotion","Industry", "Education", "Blurb", "Link"]]
FL_names

,First Name,Last Name,Title,Publish Date,New Hire/Promotion,Industry,Education,Blurb,Link
0,Charles,Harrell,Partner at Jackson Walker LLP,"December 7, 2020",New Hire,Law Practice,St. Mary's University School of Law,Corporate attorney Charles Harrell has joined ...,https://www.bizjournals.com//houston/potmsearc...
1,Bryce,Kaufman,Counsel at Latham & Watkins LLP,"November 10, 2020",Promotion,Law Practice,Stanford University Law School,Bryce D. Kaufman has been promoted to counsel ...,https://www.bizjournals.com//houston/potmsearc...
2,Kevin,Richardson,Partner at Latham & Watkins LLP,"November 10, 2020",Promotion,Law Practice,niversity of Texas School of Law,Kevin M. Richardson has been elected a partner...,https://www.bizjournals.com//houston/potmsearc...
3,Zach,Burford,Associate at Reynolds Frizzell LLP,"November 4, 2020",New Hire,Law Practice,"he University of Texas School of Law, Doctoral...",Zach Burford has joined Houston litigation fir...,https://www.bizjournals.com//houston/potmsearc...
4,Laura,S Rahman,Senior Counsel at Kean Miller LLP,"November 4, 2020",New Hire,Law Practice,NaN,Laura's practice focuses on the defense of cli...,https://www.bizjournals.com//houston/potmsearc...
5,Daniel,Dutko,"Partner at Rusty Hardin & Associates, LLP","October 29, 2020",Promotion,Law Practice,"South Texas College of Law (Houston, TX)",Mr. Dutko is a seasoned trial lawyer whose pra...,https://www.bizjournals.com//houston/potmsearc...
6,Bethany,Spare,Associate at Yetter Coleman LLP,"October 29, 2020",New Hire,Legal Services,he University of Texas School of Law,Bethany Spare represents clients in complex co...,https://www.bizjournals.com//houston/potmsearc...
7,Andrea,Hellyer,"Director Client Development at Knightscope, Inc.","October 27, 2020",New Hire,Security & Investigations,NaN,"Andrea Hellyer joined Knightscope, Inc. in Oct...",https://www.bizjournals.com//houston/potmsearc...
8,Adrienne,Randle Bond,Law Firm Shareholder at Crain Caton & James PC,"October 13, 2020",New Hire,Law Practice,olumbia School of Law,A seasoned attorney whose dynamic career spans...,https://www.bizjournals.com//houston/potmsearc...
9,Eric,Thiergod Sr,Partner at Kean Miller LLP,"October 6, 2020",New Hire,Law Practice,"South Texas College of Law (Houston, TX)",Eric is a corporate attorney whose practice in...,https://www.bizjournals.com//houston/potmsearc...


In [162]:
# Splitting the title
newTitle = FL_names["Title"].str.split(" at ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
FL_names["New Title"]= newTitle[0] 
  
# making separate last name column from new data frame 
FL_names["Firm"]= newTitle[1]
FL_names

/Users/kevinheaney/opt/anaconda3/envs/webscrape/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/kevinheaney/opt/anaconda3/envs/webscrape/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,First Name,Last Name,Title,Publish Date,New Hire/Promotion,Industry,Education,Blurb,Link,New Title,Firm
0,Charles,Harrell,Partner at Jackson Walker LLP,"December 7, 2020",New Hire,Law Practice,St. Mary's University School of Law,Corporate attorney Charles Harrell has joined ...,https://www.bizjournals.com//houston/potmsearc...,Partner,Jackson Walker LLP
1,Bryce,Kaufman,Counsel at Latham & Watkins LLP,"November 10, 2020",Promotion,Law Practice,Stanford University Law School,Bryce D. Kaufman has been promoted to counsel ...,https://www.bizjournals.com//houston/potmsearc...,Counsel,Latham & Watkins LLP
2,Kevin,Richardson,Partner at Latham & Watkins LLP,"November 10, 2020",Promotion,Law Practice,niversity of Texas School of Law,Kevin M. Richardson has been elected a partner...,https://www.bizjournals.com//houston/potmsearc...,Partner,Latham & Watkins LLP
3,Zach,Burford,Associate at Reynolds Frizzell LLP,"November 4, 2020",New Hire,Law Practice,"he University of Texas School of Law, Doctoral...",Zach Burford has joined Houston litigation fir...,https://www.bizjournals.com//houston/potmsearc...,Associate,Reynolds Frizzell LLP
4,Laura,S Rahman,Senior Counsel at Kean Miller LLP,"November 4, 2020",New Hire,Law Practice,NaN,Laura's practice focuses on the defense of cli...,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel,Kean Miller LLP
5,Daniel,Dutko,"Partner at Rusty Hardin & Associates, LLP","October 29, 2020",Promotion,Law Practice,"South Texas College of Law (Houston, TX)",Mr. Dutko is a seasoned trial lawyer whose pra...,https://www.bizjournals.com//houston/potmsearc...,Partner,"Rusty Hardin & Associates, LLP"
6,Bethany,Spare,Associate at Yetter Coleman LLP,"October 29, 2020",New Hire,Legal Services,he University of Texas School of Law,Bethany Spare represents clients in complex co...,https://www.bizjournals.com//houston/potmsearc...,Associate,Yetter Coleman LLP
7,Andrea,Hellyer,"Director Client Development at Knightscope, Inc.","October 27, 2020",New Hire,Security & Investigations,NaN,"Andrea Hellyer joined Knightscope, Inc. in Oct...",https://www.bizjournals.com//houston/potmsearc...,Director Client Development,"Knightscope, Inc."
8,Adrienne,Randle Bond,Law Firm Shareholder at Crain Caton & James PC,"October 13, 2020",New Hire,Law Practice,olumbia School of Law,A seasoned attorney whose dynamic career spans...,https://www.bizjournals.com//houston/potmsearc...,Law Firm Shareholder,Crain Caton & James PC
9,Eric,Thiergod Sr,Partner at Kean Miller LLP,"October 6, 2020",New Hire,Law Practice,"South Texas College of Law (Houston, TX)",Eric is a corporate attorney whose practice in...,https://www.bizjournals.com//houston/potmsearc...,Partner,Kean Miller LLP


In [163]:
FL_names = FL_names.drop(columns =["Title"])
FL_names

,First Name,Last Name,Publish Date,New Hire/Promotion,Industry,Education,Blurb,Link,New Title,Firm
0,Charles,Harrell,"December 7, 2020",New Hire,Law Practice,St. Mary's University School of Law,Corporate attorney Charles Harrell has joined ...,https://www.bizjournals.com//houston/potmsearc...,Partner,Jackson Walker LLP
1,Bryce,Kaufman,"November 10, 2020",Promotion,Law Practice,Stanford University Law School,Bryce D. Kaufman has been promoted to counsel ...,https://www.bizjournals.com//houston/potmsearc...,Counsel,Latham & Watkins LLP
2,Kevin,Richardson,"November 10, 2020",Promotion,Law Practice,niversity of Texas School of Law,Kevin M. Richardson has been elected a partner...,https://www.bizjournals.com//houston/potmsearc...,Partner,Latham & Watkins LLP
3,Zach,Burford,"November 4, 2020",New Hire,Law Practice,"he University of Texas School of Law, Doctoral...",Zach Burford has joined Houston litigation fir...,https://www.bizjournals.com//houston/potmsearc...,Associate,Reynolds Frizzell LLP
4,Laura,S Rahman,"November 4, 2020",New Hire,Law Practice,NaN,Laura's practice focuses on the defense of cli...,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel,Kean Miller LLP
5,Daniel,Dutko,"October 29, 2020",Promotion,Law Practice,"South Texas College of Law (Houston, TX)",Mr. Dutko is a seasoned trial lawyer whose pra...,https://www.bizjournals.com//houston/potmsearc...,Partner,"Rusty Hardin & Associates, LLP"
6,Bethany,Spare,"October 29, 2020",New Hire,Legal Services,he University of Texas School of Law,Bethany Spare represents clients in complex co...,https://www.bizjournals.com//houston/potmsearc...,Associate,Yetter Coleman LLP
7,Andrea,Hellyer,"October 27, 2020",New Hire,Security & Investigations,NaN,"Andrea Hellyer joined Knightscope, Inc. in Oct...",https://www.bizjournals.com//houston/potmsearc...,Director Client Development,"Knightscope, Inc."
8,Adrienne,Randle Bond,"October 13, 2020",New Hire,Law Practice,olumbia School of Law,A seasoned attorney whose dynamic career spans...,https://www.bizjournals.com//houston/potmsearc...,Law Firm Shareholder,Crain Caton & James PC
9,Eric,Thiergod Sr,"October 6, 2020",New Hire,Law Practice,"South Texas College of Law (Houston, TX)",Eric is a corporate attorney whose practice in...,https://www.bizjournals.com//houston/potmsearc...,Partner,Kean Miller LLP


In [164]:
# Reorder columns
FL_names = FL_names[["First Name", "Last Name", "New Title", "Firm", "Publish Date", "New Hire/Promotion","Industry", "Education", "Blurb", "Link"]]
FL_names

,First Name,Last Name,New Title,Firm,Publish Date,New Hire/Promotion,Industry,Education,Blurb,Link
0,Charles,Harrell,Partner,Jackson Walker LLP,"December 7, 2020",New Hire,Law Practice,St. Mary's University School of Law,Corporate attorney Charles Harrell has joined ...,https://www.bizjournals.com//houston/potmsearc...
1,Bryce,Kaufman,Counsel,Latham & Watkins LLP,"November 10, 2020",Promotion,Law Practice,Stanford University Law School,Bryce D. Kaufman has been promoted to counsel ...,https://www.bizjournals.com//houston/potmsearc...
2,Kevin,Richardson,Partner,Latham & Watkins LLP,"November 10, 2020",Promotion,Law Practice,niversity of Texas School of Law,Kevin M. Richardson has been elected a partner...,https://www.bizjournals.com//houston/potmsearc...
3,Zach,Burford,Associate,Reynolds Frizzell LLP,"November 4, 2020",New Hire,Law Practice,"he University of Texas School of Law, Doctoral...",Zach Burford has joined Houston litigation fir...,https://www.bizjournals.com//houston/potmsearc...
4,Laura,S Rahman,Senior Counsel,Kean Miller LLP,"November 4, 2020",New Hire,Law Practice,NaN,Laura's practice focuses on the defense of cli...,https://www.bizjournals.com//houston/potmsearc...
5,Daniel,Dutko,Partner,"Rusty Hardin & Associates, LLP","October 29, 2020",Promotion,Law Practice,"South Texas College of Law (Houston, TX)",Mr. Dutko is a seasoned trial lawyer whose pra...,https://www.bizjournals.com//houston/potmsearc...
6,Bethany,Spare,Associate,Yetter Coleman LLP,"October 29, 2020",New Hire,Legal Services,he University of Texas School of Law,Bethany Spare represents clients in complex co...,https://www.bizjournals.com//houston/potmsearc...
7,Andrea,Hellyer,Director Client Development,"Knightscope, Inc.","October 27, 2020",New Hire,Security & Investigations,NaN,"Andrea Hellyer joined Knightscope, Inc. in Oct...",https://www.bizjournals.com//houston/potmsearc...
8,Adrienne,Randle Bond,Law Firm Shareholder,Crain Caton & James PC,"October 13, 2020",New Hire,Law Practice,olumbia School of Law,A seasoned attorney whose dynamic career spans...,https://www.bizjournals.com//houston/potmsearc...
9,Eric,Thiergod Sr,Partner,Kean Miller LLP,"October 6, 2020",New Hire,Law Practice,"South Texas College of Law (Houston, TX)",Eric is a corporate attorney whose practice in...,https://www.bizjournals.com//houston/potmsearc...


In [165]:
file_to_output = Path("Data_Scraped/Contacts_Final.csv")

In [166]:
# Saving the dataframe to a csv
df_to_csv = FL_names.to_csv(file_to_output)

## Searching Bing for website and emails

In [42]:
FL

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"December 7, 2020",New Hire
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"November 10, 2020",Promotion
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"November 10, 2020",Promotion
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"November 4, 2020",New Hire
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"November 4, 2020",New Hire
5,Daniel Dutko,https://www.bizjournals.com//houston/potmsearc...,"Partner at Rusty Hardin & Associates, LLP",Mr. Dutko is a seasoned trial lawyer whose pra...,"October 29, 2020",Promotion
6,Bethany Spare,https://www.bizjournals.com//houston/potmsearc...,Associate at Yetter Coleman LLP,Bethany Spare represents clients in complex co...,"October 29, 2020",New Hire
7,Andrea Hellyer,https://www.bizjournals.com//houston/potmsearc...,"Director Client Development at Knightscope, Inc.","Andrea Hellyer joined Knightscope, Inc. in Oct...","October 27, 2020",New Hire
8,Adrienne Randle Bond,https://www.bizjournals.com//houston/potmsearc...,Law Firm Shareholder at Crain Caton & James PC,A seasoned attorney whose dynamic career spans...,"October 13, 2020",New Hire
9,Eric Thiergod Sr,https://www.bizjournals.com//houston/potmsearc...,Partner at Kean Miller LLP,Eric is a corporate attorney whose practice in...,"October 6, 2020",New Hire


In [43]:
# Set the URL to bing API
search_url = "https://api.bing.microsoft.com/v7.0/search"

In [ ]:
search_term = "Bethany Spare Associate at Yetter Coleman LLP"

In [ ]:
search_results['webPages']['value'][0]['url']

In [169]:
FL_names['Bing Search Link'] = ''
FL_names

/Users/kevinheaney/opt/anaconda3/envs/webscrape/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,First Name,Last Name,New Title,Firm,Publish Date,New Hire/Promotion,Industry,Education,Blurb,Link,Bing Search Link
0,Charles,Harrell,Partner,Jackson Walker LLP,"December 7, 2020",New Hire,Law Practice,St. Mary's University School of Law,Corporate attorney Charles Harrell has joined ...,https://www.bizjournals.com//houston/potmsearc...,
1,Bryce,Kaufman,Counsel,Latham & Watkins LLP,"November 10, 2020",Promotion,Law Practice,Stanford University Law School,Bryce D. Kaufman has been promoted to counsel ...,https://www.bizjournals.com//houston/potmsearc...,
2,Kevin,Richardson,Partner,Latham & Watkins LLP,"November 10, 2020",Promotion,Law Practice,niversity of Texas School of Law,Kevin M. Richardson has been elected a partner...,https://www.bizjournals.com//houston/potmsearc...,
3,Zach,Burford,Associate,Reynolds Frizzell LLP,"November 4, 2020",New Hire,Law Practice,"he University of Texas School of Law, Doctoral...",Zach Burford has joined Houston litigation fir...,https://www.bizjournals.com//houston/potmsearc...,
4,Laura,S Rahman,Senior Counsel,Kean Miller LLP,"November 4, 2020",New Hire,Law Practice,NaN,Laura's practice focuses on the defense of cli...,https://www.bizjournals.com//houston/potmsearc...,
5,Daniel,Dutko,Partner,"Rusty Hardin & Associates, LLP","October 29, 2020",Promotion,Law Practice,"South Texas College of Law (Houston, TX)",Mr. Dutko is a seasoned trial lawyer whose pra...,https://www.bizjournals.com//houston/potmsearc...,
6,Bethany,Spare,Associate,Yetter Coleman LLP,"October 29, 2020",New Hire,Legal Services,he University of Texas School of Law,Bethany Spare represents clients in complex co...,https://www.bizjournals.com//houston/potmsearc...,
7,Andrea,Hellyer,Director Client Development,"Knightscope, Inc.","October 27, 2020",New Hire,Security & Investigations,NaN,"Andrea Hellyer joined Knightscope, Inc. in Oct...",https://www.bizjournals.com//houston/potmsearc...,
8,Adrienne,Randle Bond,Law Firm Shareholder,Crain Caton & James PC,"October 13, 2020",New Hire,Law Practice,olumbia School of Law,A seasoned attorney whose dynamic career spans...,https://www.bizjournals.com//houston/potmsearc...,
9,Eric,Thiergod Sr,Partner,Kean Miller LLP,"October 6, 2020",New Hire,Law Practice,"South Texas College of Law (Houston, TX)",Eric is a corporate attorney whose practice in...,https://www.bizjournals.com//houston/potmsearc...,


In [174]:
for i in Final_List:
    print(i["Name"] + ' ' + i["Title"])

Charles Harrell Partner at Jackson Walker LLP
Bryce Kaufman Counsel at Latham & Watkins LLP
Kevin Richardson Partner at Latham & Watkins LLP
Zach Burford Associate at Reynolds Frizzell LLP
Laura S Rahman Senior Counsel at Kean Miller LLP
Daniel Dutko Partner at Rusty Hardin & Associates, LLP
Bethany Spare Associate at Yetter Coleman LLP
Andrea Hellyer Director Client Development at Knightscope, Inc.
Adrienne Randle Bond Law Firm Shareholder at Crain Caton & James PC
Eric Thiergod Sr Partner at Kean Miller LLP
Charles M Rosson Partner at Gibbs & Bruns LLP
Heath Trisdale htrisdale@kslaw.com at King & Spalding LLP
Michael Hamilton Partner at King & Spalding LLP
Travis Clardy Of Counsel at Holland & Knight LLP
David Peterman Partner at Gray Reed
Jonathan Newton Partner at King & Spalding LLP
John Farrell Partner at Arcadi Jackson LLP
Connie Pfeiffer Partner at Yetter Coleman LLP
Kathlyn Hufstetler Senior Associate Attorney at Patel | Gaines
Sammy Ford IV Partner at AZA Law Firm
Colleen Mig

In [22]:
# for i in Final_List:
#     try:
#         search_url = "https://api.bing.microsoft.com/v7.0/search"
#         search_term = i["Name"] + ' ' + i["Title"]
#         headers = {"Ocp-Apim-Subscription-Key": subscription_key}
#         params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
#         response = requests.get(search_url, headers=headers, params=params)
#         response.raise_for_status()
#         search_results = response.json()
#         i['Bing Search Link'] = search_results['webPages']['value'][0]['url']
#         print(f"Found {i['Name']}!")
#     except:
#         print(f"Could not find {i['Name']}")
#         print(subscription_key)
#         print(response)
#     t.sleep(1)

In [181]:
# for i in Final_List:
#     try:
#         search_url = "https://api.bing.microsoft.com/v7.0/search"
#         search_term = i["Name"] + ' ' + i["Title"]
#         headers = {"Ocp-Apim-Subscription-Key": subscription_key}
#         params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
#         response = requests.get(search_url, headers=headers, params=params)
#         response.raise_for_status()
#         search_results = response.json()
#         i['Bing Search Link'] = search_results['webPages']['value'][0]['url']
#         print(f"Found {i['Name']}!")
#     except:
#         print(f"Could not find {i['Name']}")
#     t.sleep(1)

In [23]:
# Final_List

In [24]:
# FL = pd.DataFrame(Final_List)
# FL.head()

In [47]:
# FL.isna().sum()

Name                  0
Link                  0
Title                 0
Blurb                 1
Publish Date          1
New Hire/Promotion    1
Bing Search Link      0
dtype: int64

In [25]:
# FL.Title.value_counts()

In [26]:
# Saving the dataframe to a csv
# df_to_csv = FL.to_csv(file_to_output)

In [ ]:
#Convert Bing Links to a list
# BSL = FL["Bing Search Link"].to_list()
# BSL

### Testing the email search

In [ ]:
# # starting url. replace google with your own url.
# starting_url = 'https://www.kslaw.com/people?locale=en&office_id=4'

# # a queue of urls to be crawled
# unprocessed_urls = deque([starting_url])

# # set of already crawled urls for email
# processed_urls = set()

# # a set of fetched emails
# emails = set()

# # process urls one by one from unprocessed_url queue until queue is empty
# while len(unprocessed_urls):

#     # move next url from the queue to the set of processed urls
#     url = unprocessed_urls.popleft()
#     processed_urls.add(url)

#     # extract base url to resolve relative links
#     parts = urlsplit(url)
#     base_url = "{0.scheme}://{0.netloc}".format(parts)
#     path = url[:url.rfind('/')+1] if '/' in parts.path else url

#     # get url's content
#     print("Crawling URL %s" % url)
#     try:
#         response = requests.get(url)
#     except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
#         # ignore pages with errors and continue with next url
#         continue

#     # extract all email addresses and add them into the resulting set
#     # You may edit the regular expression as per your requirement
#     new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
#     emails.update(new_emails)
#     print(emails)
#     # create a beutiful soup for the html document
#     soup = BeautifulSoup(response.text, 'lxml')
#     # Once this document is parsed and processed, now find and process all the anchors i.e. linked urls in this document
#     for anchor in soup.find_all("a"):
#         # extract link url from the anchor
#         link = anchor.attrs["href"] if "href" in anchor.attrs else ''
#         # resolve relative links (starting with /)
#         if link.startswith('/'):
#             link = base_url + link
#         elif not link.startswith('http'):
#             link = path + link
#         # add the new url to the queue if it was not in unprocessed list nor in processed list yet
#         if not link in unprocessed_urls and not link in processed_urls:
#             unprocessed_urls.append(link)

In [170]:
# for i in Final_List:
# #     print(i["Bing Search Link"])
#     response = requests.get(i["Bing Search Link"])
#     print(response)

KeyError: 'Bing Search Link'

In [27]:
# for i in Final_List:
#     emails = set()
#     try:
#         response = requests.get(i["Bing Search Link"])
#         new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
#         emails.update(new_emails)
#         i['Bing Emails'] = emails
#         if emails == set():
#             print(f'Set empty on {i["Name"]}')
#             emails = set()
#         else:
#             print(f"Successfully captured an email for {i['Name']}")
#             emails = set()
#     except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
#         # ignore pages with errors and continue with next url
#         print(f"Could not find {i['Name']}")
#         continue
# print(f"Finished all email grabs")

In [28]:
# Final_List

In [66]:
# FL = pd.DataFrame(Final_List)
# FL.head()

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Bing Search Link,Bing Emails
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"Nov 27, 2020",\nNew Hire\n,https://www.jw.com/news/welcome-charles-harrel...,{}
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"Nov 27, 2020",\nPromotion\n,https://www.lw.com/people/bryce-kaufman,{bryce.kaufman@lw.com}
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"Nov 27, 2020",\nPromotion\n,https://www.lw.com/people/kevin-richardson,{kevin.richardson@lw.com}
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"Nov 27, 2020",\nNew Hire\n,https://reynoldsfrizzell.com/attorneys/zach-bu...,{zburford@reynoldsfrizzell.com}
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"Nov 27, 2020",\nNew Hire\n,https://www.keanmiller.com/laura-s-rahman.html,{}


In [69]:
# FL['Bing Emails'] = FL['Bing Emails'].astype(str)

In [72]:
# FL["Bing Emails"] = FL["Bing Emails"].str.strip('{}')
# FL["New Hire/Promotion"] = FL["New Hire/Promotion"].str.strip('\n')


In [73]:
# FL

,Name,Link,Title,Blurb,Publish Date,New Hire/Promotion,Bing Search Link,Bing Emails
0,Charles Harrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Jackson Walker LLP,Corporate attorney Charles Harrell has joined ...,"Nov 27, 2020",New Hire,https://www.jw.com/news/welcome-charles-harrel...,set()
1,Bryce Kaufman,https://www.bizjournals.com//houston/potmsearc...,Counsel at Latham & Watkins LLP,Bryce D. Kaufman has been promoted to counsel ...,"Nov 27, 2020",Promotion,https://www.lw.com/people/bryce-kaufman,'bryce.kaufman@lw.com'
2,Kevin Richardson,https://www.bizjournals.com//houston/potmsearc...,Partner at Latham & Watkins LLP,Kevin M. Richardson has been elected a partner...,"Nov 27, 2020",Promotion,https://www.lw.com/people/kevin-richardson,'kevin.richardson@lw.com'
3,Zach Burford,https://www.bizjournals.com//houston/potmsearc...,Associate at Reynolds Frizzell LLP,Zach Burford has joined Houston litigation fir...,"Nov 27, 2020",New Hire,https://reynoldsfrizzell.com/attorneys/zach-bu...,'zburford@reynoldsfrizzell.com'
4,Laura S Rahman,https://www.bizjournals.com//houston/potmsearc...,Senior Counsel at Kean Miller LLP,Laura's practice focuses on the defense of cli...,"Nov 27, 2020",New Hire,https://www.keanmiller.com/laura-s-rahman.html,set()
5,Daniel Dutko,https://www.bizjournals.com//houston/potmsearc...,"Partner at Rusty Hardin & Associates, LLP",Mr. Dutko is a seasoned trial lawyer whose pra...,"Nov 27, 2020",Promotion,https://www.rustyhardin.com/our-team/daniel-du...,"'ddutko@rustyhardin.com', 'info@rustyhardin.com'"
6,Bethany Spare,https://www.bizjournals.com//houston/potmsearc...,Associate at Yetter Coleman LLP,Bethany Spare represents clients in complex co...,"Nov 27, 2020",New Hire,https://www.yettercoleman.com/profiles/spare-b...,set()
7,Andrea Hellyer,https://www.bizjournals.com//houston/potmsearc...,"Director Client Development at Knightscope, Inc.","Andrea Hellyer joined Knightscope, Inc. in Oct...","Nov 27, 2020",New Hire,https://www.bizjournals.com/houston/potmsearch...,set()
8,Adrienne Randle Bond,https://www.bizjournals.com//houston/potmsearc...,Law Firm Shareholder at Crain Caton & James PC,A seasoned attorney whose dynamic career spans...,"Nov 27, 2020",New Hire,https://www.craincaton.com/attorneys/adrienne-...,"'info@craincaton.com', 'abond@craincaton.com'"
9,Eric Thiergod Sr,https://www.bizjournals.com//houston/potmsearc...,Partner at Kean Miller LLP,Eric is a corporate attorney whose practice in...,"Nov 27, 2020",New Hire,https://www.keanmiller.com/eric-r-thiergood-sr...,set()


In [74]:
# # Saving the dataframe to a csv
# df_to_csv = FL.to_csv(file_to_output)

In [ ]:
# Final_List

In [8]:
# edit_csv = pd.read_csv("Data_Scraped/Contacts_Final.csv")

In [29]:
# edit_csv.head()